# 1. Clova Part

In [1]:
# Clova 결과 불러오기 
import pandas as pd 
def make_clova_result(file_path):
    path = file_path
    cnt=0
    with open(path, 'r') as file:
        lines = file.readlines()

    cnt= 0
    bl = []
    for line in lines: 
        cnt+=1
        a=  line.split("\t")
        if a[0][0:4] == '/mnt':
            word = a[1]
            word = word.strip()
            bl.append(word)
        else:
            pass
        #if cnt == 10:
        #    break

    # find out [s]
    s_case = []
    for i,w in enumerate(bl):
        if w == "[s][s]":
            #print(i)
            s_case.append(i)
        else:
            pass
            # print(w)

    print(" should check there is a error case with [s][s]")
    print("s cases : ", s_case)

    # parse logit probs from the text file 
    cnt= 0
    probs = []
    for line in lines: 
        cnt+=1
        if line[0]=="#":
            #print(line[31:])
            line = line[31:].replace(", device='cuda:0')","")
            #print(line)
            probs.append(line)   
    
    # load submit sample file  

   
    submit_file = pd.read_csv("../07.Submission_Files/[ensemble][1]_ensemble_resnet50_sharpen_last.csv")
    len(submit_file)

    # --- [s] case 가 있으면 교체
    A = dict()
    for i,row in submit_file.iterrows():
        if i in s_case:
            A[i] = row['label'] 

    # insert prediction result & probs to the df
    df = submit_file.copy()
    df['label'] = bl
    df['score'] = probs

    # bit of cleansing 
    for i, row in df.iterrows():
        row['score'] = row['score'].replace("\n","")
    
    return df    
    

In [4]:
res_clova_path = "../07.Submission_Files/log_demo_result_1017-6.txt"

vgg_clova_path = "../07.Submission_Files/log_demo_result_1017-7.txt"

abinet_path = "../07.Submission_Files/[ensemble][5][abinet]_submission_sharpen_testset_conf.csv"



res_clova_df = make_clova_result(res_clova_path)
vgg_clova_df = make_clova_result(vgg_clova_path)


 should check there is a error case with [s][s]
s cases :  []
 should check there is a error case with [s][s]
s cases :  []


In [5]:
abinet_df = pd.read_csv(abinet_path)
abinet_df.columns = ['id','label','score']

In [6]:
abinet_df.rename(columns={"confidence_score" :"score"}, inplace=True)

# 2. Resnet Part

In [9]:
import pandas as pd 
respath = "../07.Submission_Files/[ensemble][1]_ensemble_resnet50_sharpen_last.csv"
respath2 = "../07.Submission_Files/[ensemble][2]resnet152_DataAdd_STR_last.csv"
resnet_1 = pd.read_csv(respath)
resnet_2 = pd.read_csv(respath2)

resnet_1[0:20]

,id,label,score
0,TEST_00000,남말,"[0.9383753, 0.9008722]"
1,TEST_00001,상활,"[0.9904261, 0.65888625]"
2,TEST_00002,받아들이다,"[0.99807173, 0.99248403, 0.9993642, 0.9995378,..."
3,TEST_00003,바구니,"[0.99996996, 0.83414507, 0.9970078]"
4,TEST_00004,살,[0.99994314]
5,TEST_00005,빼놓다,"[0.99985886, 0.9999571, 0.9998921]"
6,TEST_00006,인식하다,"[0.9956982, 0.99629384, 0.9999963, 0.99988043]"
7,TEST_00007,센터,"[0.9718113, 0.99638736]"
8,TEST_00008,소풍,"[0.99997497, 0.9999759]"
9,TEST_00009,광주,"[0.99989676, 0.94847757]"


In [10]:
# prepare two datasets
clova_1 = res_clova_df.copy()
clova_2 = vgg_clova_df.copy()

resnet_1 = resnet_1
resnet_2 = resnet_2
abinet_1 = abinet_df.copy()

def text_clear(df):
    for i,row in df.iterrows():
        row['score'] = row['score'].replace("[","").replace("]","")
    return df 

clova_1 = text_clear(clova_1)
clova_2 = text_clear(clova_2)
resnet_1 = text_clear(resnet_1)
resnet_2 = text_clear(resnet_2)
abinet_1 = text_clear(abinet_1)


In [ ]:
# testing 
import numpy as np 

er= []
ensemble_result = []
for i in range(len(clova_1)):
    clo_c   = clova_1.iloc[i]['score']
    clo_vgg = clova_2.iloc[i]['score']
    res_c   = resnet_1.iloc[i]['score']
    res_b   = resnet_2.iloc[i]['score']
    abi_a   = abinet_1.iloc[i]['score']
    

    #------
    word_c = clova_1.iloc[i]['label']
    word_v = clova_2.iloc[i]['label']
    word_r = resnet_1.iloc[i]['label']
    word_rb = resnet_2.iloc[i]['label']
    word_a = abinet_1.iloc[i]['label']


    
    C = [float(i) for i in clo_c.split(",")]
    V = [float(i) for i in clo_vgg.split(",")]
    D = [float(i) for i in res_c.split(",")]
    W = [float(i) for i in res_b.split(",")]
    A = [float(i) for i in abi_a.split(",")]
    #print("C : ",C)
    #print("V : ",V)
    #print("D : ",D)
    #print("W : ",W)
    #print("A : ",A)
    qw= []

    words_clova =   [i for i in word_c.split(",")]
    words_clova_vgg =   [i for i in word_v.split(",")]
    words_resnet =  [i for i in word_r.split(",")]
    words_resnet_152 =  [i for i in word_rb.split(",")]
    words_abinet =  [i for i in word_a.split(",")]


    word_bag = []

    cnt= 0
    for c,v,r,w,a in zip(C,V,D,W,A):
        #for z in range(len(c)):
        # print("c : ", c)
        # print("v : ", v)
        # print("r : ", r)
        # print("r : ", w)
        # print("r : ", a)
        max_idx = np.argmax([c,v,r,w,a])
        print("max_idx : ",max_idx)
        
        if max_idx ==0:
            qw.append("clova")
        elif max_idx ==1:
            qw.append("clova_vgg")
        elif max_idx ==2:
            qw.append("res")
        elif max_idx ==3:
            qw.append("res_152")
        elif max_idx ==4:
            qw.append("abinet")
        cnt+=1
        print("cnt",cnt)
        #print("qw : ", qw)
        
    er.append(qw)
    ensemble_result.append(word_bag)

In [ ]:
# from the choice list made, pick the bigger one 

#------------------------
# in the case of length different in both cases => 418 cases occur 
# option 1) clova 2) resnet 
# i selected 1) clova 
#-----------------------

clova_1['choice'] = er

cnt=0
answers = []
for i in range(len(clova_1)):
    #------
    word_c = clova_1.iloc[i]['label']
    word_v = clova_2.iloc[i]['label']
    word_r = resnet_1.iloc[i]['label']
    word_rb = resnet_2.iloc[i]['label']
    word_a = abinet_1.iloc[i]['label']
    
      
    choice = clova_1.iloc[i]['choice']

    words = []

    #max_length = max( [len(i) for i in [word_c,word_v,word_r,word_rb,word_a]])
    for k in range(len(word_a)): # 길이가 다른 경우를 설정해주려면 word_a => 값으로 변경해줘야함. 
        
        try:
            if choice[k] == 'clova':
                words.append(word_c[k])
            elif choice[k] =='clova_vgg':
                words.append(word_v[k])
            elif choice[k] == 'res':
                words.append(word_r[k])
            elif choice[k] == 'res_152':
                words.append(word_rb[k])
            elif choice[k] == 'abinet':
                words.append(word_a[k])
        except:
            print("diff length")
            # 길이가 다른 경우 clova_res 쪽 정답을 일단 채택 => 변경 avg max confi 
            print(f"this is append {word_a[k]}")
            words.append(word_a[k])
            cnt+=1
    words = "".join(words)
    answers.append(words)

print("wrong cases ! : ", cnt)



In [ ]:
# let's pull the all the merged result to dataFrame ! 
clova_1['label'] = answers
#del df['score']

In [ ]:
# length validation 
aaa =[] 
for i,row in clova_1.iterrows():
    if len(row['label']) > 6:
        aaa.append(i)
print(aaa) 

In [ ]:
del clova_1['score']
del clova_1['choice']

In [ ]:
clova_1.to_csv("../[ensemble_result]clova[res,vgg]_res[50,152]_abinet.csv",index=False)